# Decision Tree Object Classifier
## Loading Libraries

In [3]:
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_wine
np.set_printoptions(precision=4)
sns.set()
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# Data preprocessing and machine learning
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
# To measure performance
from sklearn import metrics

## Dataset

In [105]:
# SQUARE
with open('../Control/data_pos_trial.json') as f:
    full_data_dict_square = json.load(f)
# HEXAGON
with open('../Control/data_pos_trial_hexagon.json') as f:
    full_data_dict_hexagon = json.load(f)
# CIRCLE
with open('../Control/data_pos_trial_circle.json') as f:
    full_data_dict_circle = json.load(f)
# RECTANGLE
with open('../Control/data_pos_trial_rectangle.json') as f:
    full_data_dict_rectangle = json.load(f)

number_trials = len(full_data_dict_square) + len(full_data_dict_rectangle) 
number_trials += len(full_data_dict_circle) + len(full_data_dict_hexagon) 
data_array  = np.empty((0,142), int) # 5 --> LF1_2;TIME_12;CLASS
data_class = []

poses = [60,65,70]
gaps = [34,42,50]
dims = [20,25,30]
trials = [1,2]
data_list=[]

# class --> 0:square 1:hexagon 2:circle 3:rectangle
# square and hexagon same trials
for trial in trials:
    for gap in gaps:
        for dim in dims:
            for pose in poses:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+"_"+str(trial)
                data_dict_square = full_data_dict_square[dict_key]
                data_trial = np.append(np.array(data_dict_square['RF_motion1'][0]),
                                       np.array(data_dict_square['LF_motion2'][0]))[:142]
                data_trial = data_trial[np.newaxis,:]
                data_class += [0]
                data_array = np.append(data_array, data_trial, axis = 0)
                data_dict_hexagon = full_data_dict_hexagon[dict_key]
                data_trial = np.append(np.array(data_dict_hexagon['RF_motion1'][0]),
                                       np.array(data_dict_hexagon['LF_motion2'][0]))[:142]
                data_trial = data_trial[np.newaxis,:]
                data_class += [1]
                data_array = np.append(data_array, data_trial,axis = 0)
# for circle the dim 20 gaps 50 only have the 70 pose
dims_circle = [20,25]
for trial in trials:
    for gap in gaps:
        for dim in dims_circle:
            for pose in poses:
                if not ((dim == 20 and gap == 50 and pose == 60) or 
                    (dim == 20 and gap == 50 and pose == 65)):
                    dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+"_"+str(trial)
                    data_dict_circle = full_data_dict_circle[dict_key]
                    data_trial = np.append(np.array(data_dict_circle['RF_motion1'][0]),
                                           np.array(data_dict_circle['LF_motion2'][0]))[:142]
                    data_trial = data_trial[np.newaxis, :]
                    data_class += [2]
                    data_array = np.append(data_array, data_trial,axis = 0)
# for rectangle there is only one object of dim 20x25
dims_rectangle = [[20,25],[25,20]]
for trial in trials:
    for gap in gaps:
        for dim in dims_rectangle:
            for pose in poses:
                dict_key = "dim_"+str(dim[0])+"_"+str(dim[1])+"_pose_"+str(pose)+"_gap_"+str(gap)+"_"+str(trial)
                data_dict_rectangle = full_data_dict_rectangle[dict_key]
                data_trial = np.append(np.array(data_dict_rectangle['RF_motion1'][0]),
                                       np.array(data_dict_rectangle['LF_motion2'][0]))[:142]
                data_trial = data_trial[np.newaxis, :]
                data_class += [3]
                data_array = np.append(data_array, data_trial,axis = 0)


# Random Forest Classifier
https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76

In [145]:
# Splitting Dataset
y = data_class
X = data_array
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 18, random_state = 100) #10% of the train data_Set

# Number of features 
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=1, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(X_train, y_train)

# Actual class predictions
rf_predictions = model.predict(X_test)
print("the prection of the classification is :")
print(y_test)
print(rf_predictions)
# Probabilities for each class
rf_probs = model.predict_proba(X_test)
print("the probability for the classification is :")
print(rf_probs)

from sklearn.metrics import roc_auc_score

# Calculate roc auc with "ovr"
roc_value_ovr = roc_auc_score(y_test, rf_probs, multi_class='ovr')
# Calculate roc auc with "ovo"
roc_value_ovo = roc_auc_score(y_test, rf_probs, multi_class='ovo')

the prection of the classification is :
[1, 0, 0, 3, 1, 1, 3, 0, 0, 1, 1, 1, 2, 0, 2, 3, 3, 0]
[1 0 0 3 1 1 3 0 0 2 1 2 2 0 2 3 3 0]
the probability for the classification is :
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]
